# 麗寶旅遊
- 抓網址

In [42]:
def get_url(urlfirst, urlnext):
    import requests
    from bs4 import BeautifulSoup as bs
    import time

    payload={
    'debug':'',
    'GO':'GO',
    'AIR_PT':'',
    'JOIN_TP':'1',
    'TRAREA':'0006',
    'TRNATN':'JP',
    'TRCITY':'',
    's_year':'2016',
    's_month':'06',
    's_day':'01',
    'e_year':'2016',
    'e_month':'12',
    'e_day':'31',
    't_days':'',
    'bodjet':'',
    'ITN_RK':'',
    'CARR_CD':'',
    'frm':'searcheng',
    'ikeyword':'',
    'hITN_AREA':'',
    'hITN_NATN':'',
    'hITN_CITY':'',
    'TWFG':'N',
    'chkForm':'ok',
    'x':'15',
    'y':'13'
    }

    rs = requests.session()  #無痕模式
    res = rs.post(urlfirst, data = payload)  #該網頁是POST,要加payload
    res.encoding = 'big5' #轉換編碼
    soup = bs(res.text, 'lxml')
    numpage =  int(soup.select('td.go-contant-row font')[0].text.split()[1]) #頁數
    for i in range (1, numpage + 1):
        res2 = rs.get(urlnext.format(i))
        soup2 = bs(res2.text, 'lxml')
        for ele in soup2.select('a.go-contant-link01'):
            print 'http://www.visatour.com.tw/' + ele['href']
            get_data('http://www.visatour.com.tw/' + ele['href'])
            time.sleep(1)

- 細項資料

In [43]:
def get_data(url):
    import requests
    from bs4 import BeautifulSoup as bs
    import re
    from selenium import webdriver
    import json
    
    global tourlist  #tourlist設為global為全域變數
    global num   #num設為global為全域變數
    
    dic={"agency":"", "title":"", "prodNo1":"", "prodNo2":"", "tour":{}, "href":""}
    
    rs = requests.session()
    res = rs.get(url)
    res.encoding = 'big5'  #編碼轉換成big5
    soup = bs(res.text, 'lxml')
    num += 1  #計算總共行程數

    #title  利用phantomjs_path抓title
    phantomjs_path = r'C:\Python27\Lib\phantomjs-2.1.1-windows\bin\phantomjs.exe'
    browser = webdriver.PhantomJS(phantomjs_path)
    browser.get(url)
    tour = browser.find_elements_by_xpath("//div[@id='Main']/div/table/tbody/tr/td/table/tbody/tr/td")
    dic["title"] = '-'.join(tour[1].text.split("-")[0:-1])

    #agency
    dic["agency"] = u"麗寶旅遊"
    
    #prodNo   將網址切割只剩下MGRUP_CD, GRUP_CD
    rear = url.strip('&SUB_CD=GO&JOIN_TP=1')  
    rearstring = rear.split('?MGRUP_CD=')
    MGRUP_CD, GRUP_CD =  rearstring[1].split('&GRUP_CD=')  
    dic["prodNo1"] = GRUP_CD
    dic["prodNo2"] = MGRUP_CD
    
    #href
    dic["href"] = url

    #tour  每天的行程不同，用day1..day+n來區分
    i=0
    for ele in soup.select('#Main table td'):
        if re.search(u'第.*天',ele.text) != None and len(ele.text)<=5:
            dic["tour"]['Day'+ str(ele.text.split()[1])] = soup.select('#Main table td')[i+2].text.strip().split()[-1]
        i=i+1
            
    tourlist.append(dic)  #將dic加入tourlist裡 

- 抓資料

In [44]:
import timeit
tourlist = list()
num = 0
start = timeit.default_timer()  #開始爬資料時間

urlfirst = 'http://www.visatour.com.tw/eweb_visatour/GO/L_GO_List.asp'
urlnext = 'http://www.visatour.com.tw/eweb_visatour/GO/l_go_list.asp?frm=L_GO_LIST&iPage={}&loc=&t_days=&bodjet=&s_date=2016/06/01&e_date=2016/12/31&REF_AMRNK=1&SUB_CD=GO&TRAREA=0006&TRNATN=JP&iOrdBy='

get_url(urlfirst,urlnext)
stop = timeit.default_timer() #結束爬資料時間


http://www.visatour.com.tw//eWeb_visatour/GO/V_GO_Detail.asp?MGRUP_CD=JOKA4CIF&GRUP_CD=JOKA4CI6603U&SUB_CD=GO&JOIN_TP=1
http://www.visatour.com.tw//eWeb_visatour/GO/V_GO_Detail.asp?MGRUP_CD=JOOA5-BRK10&GRUP_CD=JOOA5BR6603K&SUB_CD=GO&JOIN_TP=1
http://www.visatour.com.tw//eWeb_visatour/GO/V_GO_Detail.asp?MGRUP_CD=JSPK5-CIK1&GRUP_CD=JSPK5CI6604K&SUB_CD=GO&JOIN_TP=1
http://www.visatour.com.tw//eWeb_visatour/GO/V_GO_Detail.asp?MGRUP_CD=JOOA5-A&GRUP_CD=JOOA5GE6604A&SUB_CD=GO&JOIN_TP=1
http://www.visatour.com.tw//eWeb_visatour/GO/V_GO_Detail.asp?MGRUP_CD=JOOA5-B&GRUP_CD=JOOA5GE6604B&SUB_CD=GO&JOIN_TP=1
http://www.visatour.com.tw//eWeb_visatour/GO/V_GO_Detail.asp?MGRUP_CD=JSPK5-01GE&GRUP_CD=JSPK5GE6605A&SUB_CD=GO&JOIN_TP=1
http://www.visatour.com.tw//eWeb_visatour/GO/V_GO_Detail.asp?MGRUP_CD=JHIJ5-CIK3&GRUP_CD=JHIJ5CI6605K&SUB_CD=GO&JOIN_TP=1
http://www.visatour.com.tw//eWeb_visatour/GO/V_GO_Detail.asp?MGRUP_CD=JSPK5-01GE&GRUP_CD=JSPK5GE6606A&SUB_CD=GO&JOIN_TP=1
http://www.visatour.com.tw//eWe

- 計算所花時間和檔案數目

In [45]:
print '共花了' ,stop - start, '秒'
print '共有' , num , '檔案'

共花了 1867.03768916 秒
共有 312 檔案


- 讀取list內容

In [46]:
import json
print json.dumps(tourlist, encoding='UTF-8', ensure_ascii=False)

[{"title": "沖繩好好玩HI翻沖繩四日～美國村海洋博公園古宇利大橋(早早)", "agency": "麗寶旅遊", "tour": {"Day4": "沖繩／台北", "Day2": "名護ORION啤酒廠－海洋博公園(水族館、海豚表演)－古宇利大橋－古宇利海洋塔", "Day3": "琉球王朝古蹟【守禮門、首里城(外城)】－免稅店－藥妝店－玉泉洞王國村＋琉球大鼓隊表演－OUTLET名牌購物商圈－自行前往國際通逛街", "Day1": "RYCOM"}, "href": "http://www.visatour.com.tw//eWeb_visatour/GO/V_GO_Detail.asp?MGRUP_CD=JOKA4CIF&GRUP_CD=JOKA4CI6603U&SUB_CD=GO&JOIN_TP=1", "prodNo1": "JOKA4CI6603", "prodNo2": "JOKA4CIF"}, {"title": "關西溫泉美食～京都、奈良、神戶、大阪五日遊", "agency": "麗寶旅遊", "tour": {"Day4": "住宿飯店－古都傳統建築∼格子之家－世界文化遺產．巍峨大佛∼東大寺－悠遊自在梅花鹿群∼奈良公園－豐臣秀吉雄霸天下的歷史舞台∼大阪城（不上天守閣）－大阪免稅店－美食＆購物天堂∼道頓堀＋心齋橋－大阪", "Day5": "住宿飯店－關西國際空港／高雄", "Day2": "住宿飯店－千年古剎．文化遺產∼清水寺－石坂小徑．古都風情∼二年坂、三年坂－有趣的三猿像（非禮勿視、言、聽）∼八坂庚申堂－華麗織錦的演出∼西陣織會館（和服秀）－日本第一銅鑄朱紅大鳥居∼平安神宮－住宿飯店", "Day3": "住宿飯店－琵琶湖畔散策－比叡山國立公園∼延曆寺（不滅之燈）－★米其林旅遊評選三顆星景點★∼宇治平等院－朱紅色千本鳥居∼伏見稻荷大社－住宿飯店", "Day1": "高雄／大阪關西國際機場－沈醉在花與鳥的天堂樂園∼神戶花鳥園－明治時期西洋建築獨特風格∼北野異人坂－莫塞克廣場－神戶"}, "href": "http://www.visatour.com.tw//eWeb_visatour/GO/V_GO_Detail.asp?MGRUP_CD=JOOA5-BRK10&GRUP_CD=JOOA5BR6603K&SUB_CD=GO&JO

- 寫入到檔案中

In [47]:
import json
from json import load
diction = {'response':tourlist}
with open('LiBow.json', 'w') as f:
    json.dump(diction, f)